In [1]:
!pip install pandas
!pip install requests

import pandas as pd
import unicodedata
import datetime

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
url_componentes = 'componentes-graduacao.csv'
url_unidades = 'http://dados.ufrn.br/dataset/da6451a5-1a59-4630-bdc2-97f6be4a59c2/resource/3f2e4e32-ef1a-4396-8037-cbc22a89d97f/download/unidades.csv'
url_docentes = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'
url_avaliacoes = 'http://dados.ufrn.br/dataset/d5723d75-7e6e-4264-82aa-b96909b69f63/resource/7accd1d2-2793-460e-b98d-87a0679b9155/download/avaliacaodocencia.csv'

In [3]:
df_unidades = pd.read_csv(url_unidades, ';')
df_unidades = df_unidades[['id_unidade_responsavel', 'unidade_responsavel']]
df_unidades.rename(columns={'id_unidade_responsavel': 'id_unidade', 'unidade_responsavel': 'lotacao'}, inplace=True)
df_unidades = df_unidades[(df_unidades['lotacao'] == 'DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA') | (df_unidades['lotacao'] == 'INSTITUTO METROPOLE DIGITAL')].drop_duplicates()
df_unidades.reset_index(inplace=True, drop=True)
df_unidades.head()

,id_unidade,lotacao
0,125,DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA
1,6069,INSTITUTO METROPOLE DIGITAL


In [4]:
df_docentes = pd.read_csv(url_docentes, ';')
df_docentes.columns

Index([u'id_servidor', u'siape', u'nome', u'formacao',
       u'tipo_jornada_trabalho', u'vinculo', u'categoria', u'classe_funcional',
       u'id_unidade_lotacao', u'lotacao', u'admissao', u'Unnamed: 11'],
      dtype='object')

In [5]:
df_docentes = df_docentes[df_docentes['categoria'] == 'PROFESSOR DO MAGISTERIO SUPERIOR']

In [6]:
df_docentes.rename(columns={'id_servidor': 'id_docente', 'admissao': 'data_admissao', 'id_unidade_lotacao': 'id_unidade'}, inplace=True)
df_docentes = df_docentes.merge(df_unidades[['id_unidade']], on='id_unidade', how='inner')
df_docentes.rename(columns={'id_unidade': 'unidade'}, inplace=True)
df_docentes['data_admissao'] = [ data.date() for data in pd.to_datetime(df_docentes['data_admissao'])]
df_docentes = df_docentes[['id_docente', 'nome', 'formacao', 'unidade', 'data_admissao']]
df_docentes.head()

,id_docente,nome,formacao,unidade,data_admissao
0,5755281,ADJA FERREIRA DE ANDRADE,DOUTORADO,6069,2012-11-01
1,5755819,ANDERSON PAIVA CRUZ,DOUTORADO,6069,2013-04-25
2,5757969,ANDRE LUIZ DA SILVA SOLINO,MESTRADO,6069,2016-01-19
3,5759298,ANNA GISELLE CAMARA DANTAS RIBEIRO RODRIGUES,DOUTORADO,6069,2017-05-26
4,3032089,APUENA VIEIRA GOMES,DOUTORADO,6069,2006-08-25


In [7]:
df_componentes = pd.read_csv(url_componentes, sep=',')
df_componentes = df_componentes[(df_componentes['unidade_responsavel'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA') | (df_componentes['unidade_responsavel'] == 'INSTITUTO METROPOLE DIGITAL')]
df_componentes.head()

,id_componente,codigo,nivel,nome,unidade_responsavel
6,20007,DIM0072,G,TECNICAS DE ESPECIFICACAO FORMAL,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1033,21073,DIM0056,G,SOFTWARE BASICO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1043,21083,DIM0346,G,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1619,21667,DIM0046,G,ALGORITMOS E TECNICAS DE PROGRAMACAO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1620,21668,DIM0003,G,ORGANIZACAO DE COMPUTADORES,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA


In [8]:
unidades_normalizadas = [unicodedata.normalize('NFKD', unicode(nome, "utf-8")).encode('ASCII', 'ignore') for nome in df_componentes['unidade_responsavel']]
series = pd.Series(data=unidades_normalizadas, index=df_componentes.index)
df_componentes['unidade_responsavel'] = series

In [9]:
df_componentes = df_componentes.merge(df_unidades[['id_unidade', 'lotacao']], left_on='unidade_responsavel', right_on='lotacao', how='inner')

In [10]:
df_componente_curricular = df_componentes[['id_componente', 'codigo']]
df_componente_curricular.head()

,id_componente,codigo
0,20007,DIM0072
1,21073,DIM0056
2,21083,DIM0346
3,21667,DIM0046
4,21668,DIM0003


In [11]:
df_componente_lotacao = df_componentes[['codigo', 'id_unidade', 'nome']]
df_componente_lotacao.rename(columns={'id_unidade': 'unidade'}, inplace=True)
df_componente_lotacao.head()

c:\python27\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,codigo,unidade,nome
0,DIM0072,125,TECNICAS DE ESPECIFICACAO FORMAL
1,DIM0056,125,SOFTWARE BASICO
2,DIM0346,125,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...
3,DIM0046,125,ALGORITMOS E TECNICAS DE PROGRAMACAO
4,DIM0003,125,ORGANIZACAO DE COMPUTADORES


In [12]:
turma_periodo = ['http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/0d96d930-8058-4def-9044-c3ae04c1f40c/download/turmas-2013.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e7e3cf12-a29b-491a-a895-021a43819197/download/turmas-2013.2.csv',]

In [13]:
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma', 'ano', 'nivel_ensino', 'periodo', 'id_componente_curricular']) for periodo in turma_periodo]

In [14]:
df_turmas = pd.concat(frames)

In [15]:
df_turmas.rename(columns={'id_turma': 'id_turma', 'nivel_ensino': 'nivel', 'id_componente_curricular': 'id_componente'}, inplace=True)
df_turmas = df_turmas[df_turmas['nivel'] == 'GRADUAÇÃO']
df_turmas.drop_duplicates('id_turma', inplace=True)
df_turmas['periodo'] = df_turmas['periodo'].astype(int) 
df_turmas['id_componente'] = df_turmas['id_componente'].astype(int)
df_turmas = df_turmas.merge(df_componente_curricular[['id_componente']], on='id_componente', how='inner')
df_turmas.rename(columns={'id_componente': 'componente'}, inplace=True)
df_turmas.head()

,id_turma,componente,nivel,ano,periodo
0,57596647,2022854,GRADUAÇÃO,2017,1
1,57606915,2022854,GRADUAÇÃO,2017,2
2,57586681,2022854,GRADUAÇÃO,2016,2
3,1223534,2022854,GRADUAÇÃO,2014,1
4,1233980,2022854,GRADUAÇÃO,2014,2


In [16]:
df_avaliacoes = pd.read_csv(url_avaliacoes, sep=';', usecols=['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional_media', 'postura_profissional_DP', 'atuacao_profissional_media', 'atuacao_profissional_DP'])
df_avaliacoes = df_avaliacoes.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_docentes[['id_docente']].astype(int), on='id_docente', how='inner')
df_avaliacoes.rename(columns={'postura_profissional_media': 'postura_profissional', 'atuacao_profissional_media': 'atuacao_profissional', 'id_docente': 'docente', 'id_turma': 'turma'}, inplace=True)
df_avaliacoes = df_avaliacoes[['docente', 'turma', 'qtd_discentes', 'postura_profissional', 'postura_profissional_DP', 'atuacao_profissional', 'atuacao_profissional_DP']]
df_avaliacoes.head()

,docente,turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP
0,22764,1233659,17,9.09,1.57,9.00,1.60
1,22764,1233661,36,9.48,1.26,9.43,1.20
2,22764,1242644,34,9.41,1.04,8.86,1.73
3,22764,57566712,36,9.81,0.51,9.48,1.35
4,22764,57566818,14,9.69,0.48,8.13,2.44


In [17]:
matriculas = [
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/e974792c-b557-470c-bf3d-ede7d5b5e6a6/download/matricula-componente-20142.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/7081446d-39f9-4374-ad0b-86ecab97e569/download/matricula-componente-20141.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/517ed5f6-f8a2-40fd-826b-6ed3388f6e88/download/matricula-componente-20132.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/674de4cc-1fc0-4314-9f04-a38f0e1f9225/download/matricula-componente-20131.csv',
]

In [18]:
dfs_matricula = [pd.read_csv(matricula, sep=';') for matricula in matriculas]

In [19]:
df_matriculas = pd.concat(dfs_matricula)

c:\python27\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [20]:
df_matriculas = df_matriculas[~df_matriculas['descricao'].isin(['CUMPRIU', 'INDEFERIDO', 'EXCLUIDA'])]

In [21]:
df_matriculas.replace('APROVADO POR NOTA', 'APROVADO', inplace=True)
df_matriculas.replace(['REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 'REPROVADO POR MÉDIA E POR FALTAS', 'REPROVADO POR NOTA E FALTA', 'DESISTENCIA', 'TRANCADO', 'CANCELADO'], 'REPROVADO', inplace=True)

In [22]:
df_matriculas = df_matriculas.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_matriculas.head()

,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade
0,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.8,0.0,f,2.0
1,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.3,0.0,f,1.0
2,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.0,0.0,f,3.0
3,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,6.2,8.0,f,3.0
4,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,7.5,8.0,f,2.0


In [23]:
df_aprovados = df_matriculas[df_matriculas['descricao'] == 'APROVADO']
df_media_aprovados = df_aprovados.groupby('id_turma')['media_final'].mean().to_frame('media_aprovados').reset_index(level=0)

df_media_aprovados.head()

,id_turma,media_aprovados
0,1197615,7.010204
1,1197616,7.031250
2,1198464,6.727273
3,1198466,7.208696
4,1198499,6.451852


In [24]:
#media_turma = df_matriculas.groupby('id_turma')['media_final'].mean().to_frame('média').reset_index(level=0)
#media_turma.head()

In [25]:
df_porcentagem = df_matriculas.groupby('id_turma')['descricao'].value_counts(normalize=True).to_frame('aprovados')

In [26]:
df_porcentagem = df_porcentagem.reset_index(level=[0, 1])
df_porcentagem = df_porcentagem[~df_porcentagem['descricao'].isin(['REPROVADO'])]
df_porcentagem = df_porcentagem[['id_turma', 'aprovados']]
df_porcentagem.head()

,id_turma,aprovados
0,1197615,0.840000
2,1197616,0.578313
4,1198464,0.846154
7,1198466,0.482517
9,1198499,0.462857


In [27]:
df_avaliacoes = df_avaliacoes.merge(df_media_aprovados, left_on='turma', right_on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_porcentagem, left_on='turma', right_on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes[['docente', 'turma', 'qtd_discentes', 'postura_profissional', 'postura_profissional_DP', 'atuacao_profissional', 'atuacao_profissional_DP', 'media_aprovados', 'aprovados']]
df_avaliacoes.head()

,docente,turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP,media_aprovados,aprovados
0,22764,1233659,17,9.09,1.57,9.00,1.60,6.670588,0.739130
1,22764,1233661,36,9.48,1.26,9.43,1.20,6.842857,0.882353
2,22764,1242644,34,9.41,1.04,8.86,1.73,7.603030,0.916667
3,22764,57566712,36,9.81,0.51,9.48,1.35,7.578788,0.860870
4,22764,57566818,14,9.69,0.48,8.13,2.44,7.458333,0.837209


In [46]:
df_avaliacoes.to_csv('avaliacao.csv', sep=';', index=False)
df_componente_lotacao.to_csv('componente_lotacao.csv', sep=';', index=False)
df_componente_curricular.to_csv('componente_curricular.csv', sep=';', index=False)

df_docentes.to_csv('docente.csv', sep=';', index=False)
df_unidades.to_csv('unidade.csv', sep=';', index=False)
df_turmas.to_csv('turma.csv', sep=';', index=False)
df_avaliacoes.head()

,docente,turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP,media_aprovados,aprovados
0,22764,1233659,17,9.09,1.57,9.00,1.60,6.670588,0.739130
1,22764,1233661,36,9.48,1.26,9.43,1.20,6.842857,0.882353
2,22764,1242644,34,9.41,1.04,8.86,1.73,7.603030,0.916667
3,22764,57566712,36,9.81,0.51,9.48,1.35,7.578788,0.860870
4,22764,57566818,14,9.69,0.48,8.13,2.44,7.458333,0.837209


In [ ]:
import pandas as pd
import requests
import json

api_url = 'http://localhost:8080/'

def make_post(df, endpoint):
    records = json.loads(df.to_json(orient='records'))
    for record in records:
        request = requests.post(endpoint, data=json.dumps(record), headers={'Content-Type': 'application/json'})
        print request.text
        if not request.ok:
            break

def insert(df_url, endpoint):
    df = pd.read_csv(df_url, sep=';')
    make_post(df, endpoint)

def insert_componentes():
    df_lotacao = pd.read_csv('componente_lotacao.csv', sep=';')
    df_curricular = pd.read_csv('componente_curricular.csv', sep=';')
    df = df_lotacao.merge(df_curricular, on='codigo')
    make_post(df, api_url+'componente')

if __name__ == '__main__':
    #insert('unidade.csv', api_url+'unidade')
    #insert('docente.csv', api_url+'docente')
    #insert_componentes()
    #insert('turma.csv', api_url+'turma')
    insert('avaliacao.csv', api_url+'avaliacao')

{"id":null,"aprovados":0.7391304348,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":22764,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1233659,"componente":null},"qtd_discentes":17,"postura_profissional":9.09,"postura_profissional_DP":1.57,"atuacao_profissional":9.0,"atuacao_profissional_DP":1.6,"media_aprovados":6.6705882353}
{"id":null,"aprovados":0.8823529412,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":22764,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1233661,"componente":null},"qtd_discentes":36,"postura_profissional":9.48,"postura_profissional_DP":1.26,"atuacao_profissional":9.43,"atuacao_profissional_DP":1.2,"media_aprovados":6.8428571429}
{"id":null,"aprovados":0.9166666667,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":22764,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1242644,"componente":null},"qtd_discentes

{"id":null,"aprovados":0.35,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":23713,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1211639,"componente":null},"qtd_discentes":22,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":9.55,"atuacao_profissional_DP":1.0,"media_aprovados":6.1}
{"id":null,"aprovados":0.5172413793,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":23713,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1217953,"componente":null},"qtd_discentes":9,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":9.89,"atuacao_profissional_DP":0.49,"media_aprovados":5.82}
{"id":null,"aprovados":0.875,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":23713,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1242646,"componente":null},"qtd_discentes":7,"postura_profissional":9.92,"

{"id":null,"aprovados":0.9090909091,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757768,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57596656,"componente":null},"qtd_discentes":11,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":7.88,"atuacao_profissional_DP":1.62,"media_aprovados":6.41}
{"id":null,"aprovados":0.8955223881,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24798,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57596658,"componente":null},"qtd_discentes":21,"postura_profissional":9.81,"postura_profissional_DP":0.54,"atuacao_profissional":7.83,"atuacao_profissional_DP":2.08,"media_aprovados":7.07}
{"id":null,"aprovados":0.8955223881,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5752649,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57596658,"componente":null},"qtd_discentes":21,"p

{"id":null,"aprovados":0.4390243902,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755402,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1211641,"componente":null},"qtd_discentes":12,"postura_profissional":8.75,"postura_profissional_DP":2.01,"atuacao_profissional":6.88,"atuacao_profissional_DP":3.28,"media_aprovados":7.35}
{"id":null,"aprovados":0.7868852459,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24853,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1217954,"componente":null},"qtd_discentes":19,"postura_profissional":9.23,"postura_profissional_DP":1.81,"atuacao_profissional":7.63,"atuacao_profissional_DP":2.94,"media_aprovados":6.4875}
{"id":null,"aprovados":0.4705882353,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24853,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1228386,"componente":null},"qtd_discentes":17,"pos

{"id":null,"aprovados":0.3,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24949,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1217936,"componente":null},"qtd_discentes":8,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":9.13,"atuacao_profissional_DP":1.43,"media_aprovados":8.4666666667}
{"id":null,"aprovados":0.4335260116,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24949,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1228413,"componente":null},"qtd_discentes":54,"postura_profissional":9.71,"postura_profissional_DP":0.77,"atuacao_profissional":9.13,"atuacao_profissional_DP":1.32,"media_aprovados":7.812}
{"id":null,"aprovados":0.2608695652,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":24949,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1228415,"componente":null},"qtd_discentes":21,"postura_p

...

{"id":null,"aprovados":0.8108108108,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756592,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586679,"componente":null},"qtd_discentes":10,"postura_profissional":9.61,"postura_profissional_DP":0.98,"atuacao_profissional":9.03,"atuacao_profissional_DP":1.57,"media_aprovados":7.33}
{"id":null,"aprovados":0.7105263158,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":26012,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1211649,"componente":null},"qtd_discentes":21,"postura_profissional":9.64,"postura_profissional_DP":0.68,"atuacao_profissional":8.72,"atuacao_profissional_DP":2.07,"media_aprovados":7.2388888889}
{"id":null,"aprovados":0.8181818182,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":26012,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1223552,"componente":null},"qtd_discentes":

{"id":null,"aprovados":0.5562913907,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":26044,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57596683,"componente":null},"qtd_discentes":49,"postura_profissional":9.67,"postura_profissional_DP":0.97,"atuacao_profissional":8.39,"atuacao_profissional_DP":2.15,"media_aprovados":6.5285714286}
{"id":null,"aprovados":0.3928571429,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755388,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1234518,"componente":null},"qtd_discentes":16,"postura_profissional":9.64,"postura_profissional_DP":0.79,"atuacao_profissional":9.43,"atuacao_profissional_DP":0.92,"media_aprovados":8.8090909091}
{"id":null,"aprovados":0.1764705882,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755388,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1210338,"componente":null},"qtd_d

{"id":null,"aprovados":0.225,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":115827,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1223550,"componente":null},"qtd_discentes":12,"postura_profissional":8.92,"postura_profissional_DP":1.24,"atuacao_profissional":8.44,"atuacao_profissional_DP":2.46,"media_aprovados":7.9}
{"id":null,"aprovados":0.8048780488,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":115827,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1233846,"componente":null},"qtd_discentes":25,"postura_profissional":9.72,"postura_profissional_DP":0.77,"atuacao_profissional":9.51,"atuacao_profissional_DP":1.01,"media_aprovados":7.6363636364}
{"id":null,"aprovados":0.6486486486,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":115827,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1242639,"componente":null},"qtd_discentes":21,"post

{"id":null,"aprovados":0.9444444444,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755046,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1211690,"componente":null},"qtd_discentes":17,"postura_profissional":9.03,"postura_profissional_DP":1.49,"atuacao_profissional":9.69,"atuacao_profissional_DP":0.92,"media_aprovados":8.0}
{"id":null,"aprovados":0.9230769231,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5751871,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1233658,"componente":null},"qtd_discentes":16,"postura_profissional":9.9,"postura_profissional_DP":0.31,"atuacao_profissional":9.49,"atuacao_profissional_DP":1.25,"media_aprovados":7.84375}
{"id":null,"aprovados":0.9230769231,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755046,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1233658,"componente":null},"qtd_discentes":16,"

{"id":null,"aprovados":0.875,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5752241,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1224068,"componente":null},"qtd_discentes":16,"postura_profissional":5.38,"postura_profissional_DP":1.56,"atuacao_profissional":6.33,"atuacao_profissional_DP":3.01,"media_aprovados":8.2857142857}
{"id":null,"aprovados":0.756097561,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5752241,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1241766,"componente":null},"qtd_discentes":38,"postura_profissional":8.03,"postura_profissional_DP":1.68,"atuacao_profissional":7.25,"atuacao_profissional_DP":2.92,"media_aprovados":7.5516129032}
{"id":null,"aprovados":0.9677419355,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5752241,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1241879,"componente":null},"qtd_discente